#**Sentiment Analysis Model on E-Commerce Website**


### Install necessary packages

In [2]:
!pip install streamlit
!pip install joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 117.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 32.5 MB/s eta 0:00:00
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzlocal-5.0.1:
      Successfully uninstalled tzlocal-5.0.1


### Imports

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score
import joblib
import streamlit as st

### Download NLTK resources

In [4]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Define preprocessing function

In [5]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words("english")]
    words = [PorterStemmer().stem(word) for word in words]
    return " ".join(words)


### Load and preprocess training data

In [10]:
with open('train.ft.txt', 'r') as train_file:
    train_data = train_file.readlines()

train_comments = [re.sub(r'^__label__[0-9]+ ', '', line).strip() for line in train_data]
train_labels = [re.findall(r'^__label__([0-9]+)', line)[0] for line in train_data]


### Train the model

In [11]:
train_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('model', LogisticRegression(max_iter=100))
])
train_pipeline.fit(train_comments, train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('model', LogisticRegression())])

### Save the trained model

In [12]:
model_filename = 'sentiment_model.pkl'
joblib.dump(train_pipeline, model_filename)


['sentiment_model.pkl']

### Load and preprocess testing data

In [13]:
with open('test.ft.txt', 'r') as test_file:
    test_data = test_file.readlines()

test_comments = [re.sub(r'^__label__[0-9]+ ', '', line).strip() for line in test_data]
test_labels = [re.findall(r'^__label__([0-9]+)', line)[0] for line in test_data]


### Predict on testing data

In [14]:
test_predictions = train_pipeline.predict(test_comments)

### Calculate evaluation metrics

In [15]:
accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions, average='weighted')
recall = recall_score(test_labels, test_predictions, average='weighted')


## Streamlit app

In [16]:
st.title("E-commerce Sentiment Analysis")
comment = st.text_area("Enter your comment:")
if st.button("Analyze"):
    preprocessed_comment = preprocess_text(comment)
    sentiment = train_pipeline.predict([preprocessed_comment])[0]
    st.write("Predicted Sentiment:", sentiment)


2023-08-16 19:59:27.627 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]


### Display evaluation metrics

In [17]:
st.subheader("Model Evaluation Metrics")
st.write("Accuracy:", accuracy)
st.write("Precision:", precision)
st.write("Recall:", recall)